### Analysis of Everypol (after JSON to pd)

In [1]:
import os
import pandas as pd
import json
import ast
import openpyxl
import janitor

from utilities import process_json_files_to_matrix, clean_email_column_no_dedupe

### Assume MCAR

Where we don't have pol. emails etc., let's just assume that the data are missing at random. 

In [2]:
everypol_hibp = process_json_files_to_matrix("../data/everypol/everypol_hibp")
everypol_hibp.shape

(8536, 220)

In [3]:
everypol_hibp.head()

,Filename,000webhost,123RF,2844Breaches,500px,8fit,ABFRL,AKP,ActMobile,Acuity,...,Zacks,Zomato,Zynga,bigbasket,db8151dd,digiDirect,eThekwiniMunicipality,iMesh,ixigo,piZap
0,E.Ronnes@Tweedekamer.nl,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Nuk.Korako@parliament.govt.nz,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,stephen.greene@sen.parl.gc.ca,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,yoliswa.yako@gmail.com,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,bsanseverino@parlamento.gub.uy,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
long_everypol_hibp = pd.melt(
    everypol_hibp,
    id_vars=['Filename'],
    var_name='Breach',
    value_name='Present'
)
long_everypol_hibp.shape

(1869384, 3)

In [5]:
long_everypol_hibp.to_csv("../data/everypol_hibp.csv", index=False)

In [6]:
breaches = pd.read_csv("../data/hipb_01_2025_breaches_data.csv")
breaches.head()

,Name,Title,Domain,BreachDate,AddedDate,ModifiedDate,PwnCount,Description,LogoPath,DataClasses,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
0,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,"In approximately March 2015, the free web host...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
1,123RF,123RF,123rf.com,2020-03-22,2020-11-15T00:59:50Z,2020-11-15T01:07:10Z,8661578,"In March 2020, the stock photo site <a href=""h...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
2,126,126,126.com,2012-01-01,2016-10-08T07:46:05Z,2016-10-08T07:46:05Z,6414191,"In approximately 2012, it's alleged that the C...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'Passwords']",False,False,False,False,False,False,False
3,17Media,17,17app.co,2016-04-19,2016-07-08T01:55:03Z,2016-07-08T01:55:03Z,4009640,"In April 2016, customer data obtained from the...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Device information', 'Email addresses', 'IP ...",True,False,False,False,False,False,False
4,17173,17173,17173.com,2011-12-28,2018-04-28T04:53:15Z,2018-04-28T04:53:15Z,7485802,"In late 2011, <a href=""https://news.softpedia....",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'Passwords', 'Usernames']",False,False,False,False,False,False,False


In [7]:
breaches["DataClasses"] = breaches["DataClasses"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
all_categories = set(item for sublist in breaches["DataClasses"] for item in sublist)
expanded_df = pd.DataFrame(
    {category: breaches["DataClasses"].apply(lambda x: category in x) for category in all_categories}
)
breaches = pd.concat([breaches.drop(columns=["DataClasses"]), expanded_df], axis=1)

In [8]:
breaches_everypol_hibp = long_everypol_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_everypol_hibp.head()

,Filename,Breach,Present,Name,Title,Domain,BreachDate,AddedDate,ModifiedDate,PwnCount,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
0,E.Ronnes@Tweedekamer.nl,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False
1,Nuk.Korako@parliament.govt.nz,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False
2,stephen.greene@sen.parl.gc.ca,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False
3,yoliswa.yako@gmail.com,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False
4,bsanseverino@parlamento.gub.uy,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False


In [9]:
boolean_columns = breaches_everypol_hibp.select_dtypes(include=['bool'])
grouped_bool_counts = boolean_columns.groupby(breaches_everypol_hibp["Filename"]).sum().reset_index()
grouped_bool_counts.describe()

,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Health insurance information,Passwords,Profile photos,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0,...,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0,8536.0
mean,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,138.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,138.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
25%,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,138.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
50%,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,138.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
75%,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,138.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
max,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,138.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0


In [10]:
grouped_bool_counts.head()

,Filename,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Health insurance information,Passwords,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
0,(i)joice.george@sansad.nic.in,207,1,0,0,12,2,1,1,138,...,0,0,0,0,1,0,0,9,1,0
1,1.office@bjpanda.org,207,1,0,0,12,2,1,1,138,...,0,0,0,0,1,0,0,9,1,0
2,101bbb@naver.com,207,1,0,0,12,2,1,1,138,...,0,0,0,0,1,0,0,9,1,0
3,2016kimkj@gmail.com,207,1,0,0,12,2,1,1,138,...,0,0,0,0,1,0,0,9,1,0
4,2016kimss@gmail.com,207,1,0,0,12,2,1,1,138,...,0,0,0,0,1,0,0,9,1,0


In [11]:
pol_dat = pd.read_csv("../data/everypol/everypol_combined_legislature_data.csv", low_memory=False)
pol_dat.head()

,id,name,sort_name,email,twitter,facebook,group,group_id,area_id,area,...,n_unique_emails,person_count_legistype,ltype,url,cc,leg_start_date,lastmod,cc3,pop2024,lastmod_year
0,35ad9676-8485-4137-9a16-50f2844f3ab2,"Adhalrao Patil, Shri Shivaji","Adhalrao Patil, Shri Shivaji",shivajirao@sansad.nic.in,NaN,NaN,Shiv Sena,SS,shirur,Shirur,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
1,f1f0f31d-ddb8-4681-82c0-4484d0d26ee3,"Adhikari, Shri Deepak (Dev)","Adhikari, Shri Deepak (Dev)",adhikari.deepak@sansad.nic.in,idevadhikari,IamTheDev,All India Trinamool Congress,AITC,ghatal,Ghatal,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
2,bb68e3c6-de79-4d07-ad1f-90bb9a61ded0,"Adhikari, Shri Sisir Kumar","Adhikari, Shri Sisir Kumar",sisiradhikari76@yahoo.com,NaN,NaN,All India Trinamool Congress,AITC,kanthi,Kanthi,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
3,c9a891c3-adb6-4581-b93b-72f9c00beacc,"Adhikari, Shri Suvendu","Adhikari, Shri Suvendu",adhikari.suvendu@sansad.nic.in,NaN,NaN,All India Trinamool Congress,AITC,tamluk,Tamluk,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
4,5c22c70d-8317-4f94-97ec-2ccadd1acdf3,"Adityanath , Shri Yogi","Adityanath , Shri Yogi",yogi.adityanath@sansad.nic.in,NaN,NaN,Bharatiya Janata Party,BJP,gorakhpur,Gorakhpur,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019


In [12]:
pol_dat.shape

(25087, 35)

In [13]:
pol_dat = clean_email_column_no_dedupe(pol_dat)
pol_dat.shape

(8512, 35)

In [14]:
pol_hibp = pol_dat.merge(grouped_bool_counts, left_on="email", right_on="Filename", how="left")
pol_hibp.head()

,id,name,sort_name,email,twitter,facebook,group,group_id,area_id,area,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
0,35ad9676-8485-4137-9a16-50f2844f3ab2,"Adhalrao Patil, Shri Shivaji","Adhalrao Patil, Shri Shivaji",shivajirao@sansad.nic.in,NaN,NaN,Shiv Sena,SS,shirur,Shirur,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
1,f1f0f31d-ddb8-4681-82c0-4484d0d26ee3,"Adhikari, Shri Deepak (Dev)","Adhikari, Shri Deepak (Dev)",adhikari.deepak@sansad.nic.in,idevadhikari,IamTheDev,All India Trinamool Congress,AITC,ghatal,Ghatal,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
2,bb68e3c6-de79-4d07-ad1f-90bb9a61ded0,"Adhikari, Shri Sisir Kumar","Adhikari, Shri Sisir Kumar",sisiradhikari76@yahoo.com,NaN,NaN,All India Trinamool Congress,AITC,kanthi,Kanthi,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
3,c9a891c3-adb6-4581-b93b-72f9c00beacc,"Adhikari, Shri Suvendu","Adhikari, Shri Suvendu",adhikari.suvendu@sansad.nic.in,NaN,NaN,All India Trinamool Congress,AITC,tamluk,Tamluk,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
4,5c22c70d-8317-4f94-97ec-2ccadd1acdf3,"Adityanath , Shri Yogi","Adityanath , Shri Yogi",yogi.adityanath@sansad.nic.in,NaN,NaN,Bharatiya Janata Party,BJP,gorakhpur,Gorakhpur,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0


In [15]:
pol_hibp.describe().round(2)

,row_id,leg_start_year,row_count,n_unique_emails,person_count_legistype,lastmod,pop2024,lastmod_year,IsVerified,IsFabricated,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,8512.00,8512.00,8512.00,8512.00,8512.00,8.512000e+03,8.153000e+03,8512.00,8131.0,8131.0,...,8131.0,8131.0,8131.0,8131.0,8131.0,8131.0,8131.0,8131.0,8131.0,8131.0
mean,260.17,2014.98,293.10,226.26,533.32,1.556591e+09,1.230668e+08,2018.99,207.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
std,181.72,1.93,181.07,160.52,418.01,2.085169e+06,3.421782e+08,0.09,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.00,1997.00,31.00,31.00,31.00,1.542919e+09,8.034100e+04,2018.00,207.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
25%,124.00,2014.00,134.00,101.00,249.00,1.555982e+09,6.378654e+06,2019.00,207.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
50%,201.00,2015.00,290.00,169.00,499.00,1.557279e+09,3.124032e+07,2019.00,207.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
75%,385.00,2016.00,410.00,327.00,606.00,1.557730e+09,6.102022e+07,2019.00,207.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0
max,748.00,2019.00,686.00,646.00,1783.00,1.557904e+09,1.441720e+09,2019.00,207.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,0.0


In [16]:
columns_to_fill = list(all_categories) + ['IsVerified', 'IsFabricated', 'IsSensitive', 
                   'IsRetired', 'IsSpamList', 'IsMalware', 'IsSubscriptionFree']
pol_hibp[columns_to_fill] = pol_hibp[columns_to_fill].fillna(0)

In [17]:
pol_hibp.groupby(['country', 'legislature', 'leg_start_year']).mean(numeric_only=True).reset_index().describe().round(2)

,leg_start_year,row_id,row_count,n_unique_emails,person_count_legistype,lastmod,pop2024,lastmod_year,IsVerified,IsFabricated,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,77.00,77.00,77.00,77.00,77.00,7.700000e+01,7.100000e+01,77.00,77.00,77.00,...,77.0,77.0,77.0,77.0,77.00,77.0,77.0,77.00,77.00,77.0
mean,2014.31,346.40,207.78,134.99,416.82,1.556156e+09,4.744066e+07,2018.97,199.50,0.96,...,0.0,0.0,0.0,0.0,0.96,0.0,0.0,8.67,0.96,0.0
std,3.26,193.07,165.17,125.88,416.12,2.896721e+06,1.730941e+08,0.16,26.08,0.13,...,0.0,0.0,0.0,0.0,0.13,0.0,0.0,1.13,0.13,0.0
min,1997.00,1.00,31.00,31.00,31.00,1.542919e+09,8.034100e+04,2018.00,32.14,0.16,...,0.0,0.0,0.0,0.0,0.16,0.0,0.0,1.40,0.16,0.0
25%,2013.00,150.00,90.00,54.00,115.00,1.555055e+09,3.308847e+06,2019.00,204.91,0.99,...,0.0,0.0,0.0,0.0,0.99,0.0,0.0,8.91,0.99,0.0
50%,2015.00,365.00,158.00,99.00,267.00,1.557278e+09,1.031910e+07,2019.00,207.00,1.00,...,0.0,0.0,0.0,0.0,1.00,0.0,0.0,9.00,1.00,0.0
75%,2016.00,503.00,291.00,152.00,554.00,1.557708e+09,4.040918e+07,2019.00,207.00,1.00,...,0.0,0.0,0.0,0.0,1.00,0.0,0.0,9.00,1.00,0.0
max,2019.00,748.00,686.00,646.00,1783.00,1.557904e+09,1.441720e+09,2019.00,207.00,1.00,...,0.0,0.0,0.0,0.0,1.00,0.0,0.0,9.00,1.00,0.0


### Let's merge the rest (India, Eur.)

In [18]:
no_hibp = process_json_files_to_matrix("../data/no_hibp")
dk_hibp = process_json_files_to_matrix("../data/dk_hibp")
in_hibp = process_json_files_to_matrix("../data/india/ls_hibp")
sg_hibp = process_json_files_to_matrix("../data/sg_hibp")

bihar_hibp = process_json_files_to_matrix("../data/india/bihar_hibp")
tn_hibp = process_json_files_to_matrix("../data/india/tn_hibp")
up_hibp = process_json_files_to_matrix("../data/india/up_hibp")
hp_hibp = process_json_files_to_matrix("../data/india/hp_hibp")
del_hibp = process_json_files_to_matrix("../data/india/delhi_hibp")

In [19]:
no_hibp

,Filename,Adapt,Adobe,Apollo,BVD,Bitly,Cit0day,DemandScience,Disqus,Dodonew,...,NetEase,OnlinerSpambot,PDL,ShareThis,TAPAirPortugal,TrikSpamBotnet,Twitter200M,VerificationsIO,YouveBeenScraped,db8151dd
0,ingvild.kjerkol@stortinget.no,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,truls.vasvik@stortinget.no,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,trine.lise.sundnes@stortinget.no,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,lisa.marie.ness.klungland@stortinget.no,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,andreas.sjalg.unneland@stortinget.no,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,torbjorn.vereide@stortinget.no,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
170,per.lundteigen@stortinget.no,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
171,eigil.knutsen@stortinget.no,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
172,frank.edvard.sve@stortinget.no,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
# Make it long
long_no_hibp    = pd.melt(no_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_dk_hibp    = pd.melt(dk_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_in_hibp    = pd.melt(in_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_sg_hibp    = pd.melt(sg_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')

long_bihar_hibp = pd.melt(bihar_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_tn_hibp    = pd.melt(tn_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_up_hibp    = pd.melt(up_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_hp_hibp    = pd.melt(hp_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_del_hibp   = pd.melt(del_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')

In [21]:
(
    pd.concat([long_no_hibp, long_dk_hibp, long_in_hibp, long_sg_hibp,
    long_bihar_hibp, long_tn_hibp, long_up_hibp, long_hp_hibp, long_del_hibp])
    .to_csv("../data/scraped_pol_hibp.csv", index=False)
)

In [22]:
# Join to breaches
breaches_no = long_no_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_dk = long_dk_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_in = long_in_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_sg = long_sg_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")

breaches_bihar = long_bihar_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_tn    = long_tn_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_up    = long_up_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_hp    = long_hp_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_del   = long_del_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")

In [23]:
# Group by email and sum the bool cols
no_breach_count = breaches_no.select_dtypes(include=['bool']).groupby(breaches_no["Filename"]).sum().reset_index()
dk_breach_count = breaches_dk.select_dtypes(include=['bool']).groupby(breaches_dk["Filename"]).sum().reset_index()
in_breach_count = breaches_in.select_dtypes(include=['bool']).groupby(breaches_in["Filename"]).sum().reset_index()
sg_breach_count = breaches_sg.select_dtypes(include=['bool']).groupby(breaches_sg["Filename"]).sum().reset_index()

bihar_breach_count = breaches_bihar.select_dtypes(include=['bool']).groupby(breaches_bihar["Filename"]).sum().reset_index()
tn_breach_count = breaches_tn.select_dtypes(include=['bool']).groupby(breaches_tn["Filename"]).sum().reset_index()
up_breach_count = breaches_up.select_dtypes(include=['bool']).groupby(breaches_up["Filename"]).sum().reset_index()
hp_breach_count = breaches_hp.select_dtypes(include=['bool']).groupby(breaches_hp["Filename"]).sum().reset_index()
del_breach_count = breaches_del.select_dtypes(include=['bool']).groupby(breaches_del["Filename"]).sum().reset_index()

In [24]:
# Norway
no_parl = pd.read_csv("../data/no/no_parliament.csv")
no_parl = clean_email_column_no_dedupe(no_parl)
print(no_parl.shape)

# Denmark
wb = openpyxl.load_workbook("../data/danish_parliament_1_2025.xlsx")
ws = wb.active  
data = list(ws.values)
dk_parl = pd.DataFrame(data)
dk_parl.columns = dk_parl.iloc[0]
dk_parl = dk_parl[1:].reset_index(drop=True)
dk_parl = clean_email_column_no_dedupe(dk_parl)
print(dk_parl.shape)

# India
in_df = pd.read_csv("../data/india/ls_long.csv")
in_df = clean_email_column_no_dedupe(in_df)
print(in_df.shape)

# Singapore
sg_df = pd.read_csv("../data/sg/sg_mp.csv")
sg_df = clean_email_column_no_dedupe(sg_df)
print(sg_df.shape)

# Bihar
with open('../data/india/bihar/bihar.txt', 'r', encoding='utf-8') as file:
    lines = file.read().splitlines()
    lines = [line.split('\t') for line in lines]

bihar_df = pd.DataFrame(lines, columns=['sr_no', 'photo', 'constituency', 'name', 'gender', 'party', 'contact', 'email'])
bihar_df = clean_email_column_no_dedupe(bihar_df)
print(bihar_df.shape)

# UP
up_df = pd.read_csv("../data/india/up/up_18_mlas.csv", usecols=['email'], encoding="utf-8", quotechar='"', sep=",", engine="python")
up_df = clean_email_column_no_dedupe(up_df)
print(up_df.shape)

# HP
hp_df = pd.read_csv("../data/india/hp_14.csv")
hp_df.columns = hp_df.columns.str.lower()
hp_df = clean_email_column_no_dedupe(hp_df)
print(hp_df.shape)

# TN
tn_df = pd.read_csv("../data/india/tn/tn.csv")
tn_df.rename(columns={"Email Address": "email"}, inplace=True)
tn_df = clean_email_column_no_dedupe(tn_df)
print(tn_df.shape)

# Delhi
del_df = pd.read_csv("../data/india/delhi/delhi_7th_assembly.csv")
del_df.rename(columns={"Email": "email"}, inplace=True)
del_df = clean_email_column_no_dedupe(del_df)
print(del_df.shape)

(174, 3)
(186, 10)
(2562, 43)
(400, 7)
(241, 8)
(106, 1)
(68, 7)
(233, 6)
(43, 8)


/home/lsys/gusto/venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [25]:
# Add metadata (to be consistent with EP)
no_parl = no_parl.assign(
    country="Norway",
    cc3="NOR",
    year=2025,
    legislature="Storting",
    ltype="unicameral legislature",
).clean_names()
dk_parl = dk_parl.assign(
    country="Denmark",
    cc3="DNK",
    year=2025,
    legislature="Folketing",
    ltype="unicameral legislature",
).clean_names()
in_df = in_df.assign(
    country="India",
    cc3="IND",
    year=2025,
    legislature="Lok Sabha",
    ltype="bicameral legislature",
).clean_names()
sg_df = sg_df.assign(
    country="Singapore",
    cc3="SGP",
    year=2025,
    legislature="Parliament",
    ltype="unicameral legislature",
).clean_names()

bihar_df = bihar_df.assign(
    country="India",
    cc3="IND",
    year=2025,
    legislature="State Legislature",
    chamber="Bihar Legislature",
    ltype="bicameral legislature",
).clean_names()
tn_df = tn_df.assign(
    country="India",
    cc3="IND",
    year=2025,
    legislature="State Legislature",
    chamber="Tamil Nadu State Legislature",
    ltype="bicameral legislature",
).clean_names()
up_df = up_df.assign(
    country="India",
    cc3="IND",
    year=2025,
    legislature="State Legislature",
    chamber="UP State Legislature",
    ltype="bicameral legislature",
).clean_names()
hp_df = hp_df.assign(
    country="India",
    cc3="IND",
    year=2025,
    legislature="State Legislature",
    ltype="bicameral legislature",
    chamber="HP Legislature",
).clean_names()
del_df = del_df.assign(
    country="India",
    cc3="IND",
    year=2025,
    legislature="State Legislature",
    chamber="Delhi Legislature",
    ltype="bicameral legislature",
).clean_names()

# Renaming col
dk_parl = dk_parl.rename_column("full_name", "name")

/home/lsys/gusto/venv/lib/python3.10/site-packages/pandas_flavor/register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


In [26]:
(
    pd.concat([no_parl, dk_parl, in_df, sg_df, bihar_df, tn_df, up_df, hp_df, del_df])
    .to_csv("../data/scraped_pol_combined_legislature_data.csv", index=False)
)

In [27]:
in_df

,mpsno,initial,firstname,lastname,gender,partyfname,partysname,statename,constname,profession,...,mpfirstlastname,maritalstatus,createdat,updatedat,email_fix,country,cc3,year,legislature,ltype
0,344,Shri,NaN,A. Raja,Male,Dravida Munnetra Kazhagam,DMK,Tamil Nadu ...,Nilgiris,Advocate ...,...,NaN,NaN,NaN,NaN,raja.andimuthu@gmail.com,India,IND,2025,Lok Sabha,bicameral legislature
1,344,Shri,NaN,A. Raja,Male,Dravida Munnetra Kazhagam,DMK,Tamil Nadu ...,Nilgiris,Advocate ...,...,NaN,NaN,NaN,NaN,a.raja@sansad.nic.in,India,IND,2025,Lok Sabha,bicameral legislature
2,5175,Shri,Narayana Swamy,Abbaiah,Male,Bharatiya Janata Party,BJP,Karnataka ...,Chitradurga,Businessperson ...,...,NaN,NaN,NaN,NaN,anarayanaswamyanekal5@gmail.com,India,IND,2025,Lok Sabha,bicameral legislature
3,5175,Shri,Narayana Swamy,Abbaiah,Male,Bharatiya Janata Party,BJP,Karnataka ...,Chitradurga,Businessperson ...,...,NaN,NaN,NaN,NaN,a.narayanswamy@sansad.nic.in,India,IND,2025,Lok Sabha,bicameral legislature
4,2654,Dr.,Farooq,Abdullah,Male,Jammu and Kashmir National Conference,J&KNC,Jammu and Kashmir ...,Srinagar,Social Worker ...,...,NaN,NaN,NaN,NaN,iamfarooq80@hotmail.com,India,IND,2025,Lok Sabha,bicameral legislature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,512,Shri,V.,Vetriselvan,Male,Dravida Munnetra Kazhagam,DMK,Tamil Nadu ...,Krishnagiri,Advocate ...,...,NaN,NaN,NaN,NaN,vetriselvan@sansad.nic.in,India,IND,2025,Lok Sabha,bicameral legislature
2558,513,Smt.,Dumpa Mary,Vijayakumari,Female,Telugu Desam Party,TDP,Andhra Pradesh ...,Bhadrachalam-ST ...,Social Worker ...,...,NaN,NaN,NaN,NaN,vijayak@sansad.nic.in,India,IND,2025,Lok Sabha,bicameral legislature
2559,518,Shri,Srikanta Datta Narasimharaja,Wadiyar,Male,Indian National Congress,INC,Karnataka ...,Mysore ...,Businessperson ...,...,NaN,NaN,NaN,NaN,wadiyar@sansad.nic.in,India,IND,2025,Lok Sabha,bicameral legislature
2560,521,Dr.(Smt.),Sudha,Yadav,Female,Bharatiya Janata Party,BJP,Haryana ...,Mahendergarh ...,Educationist ...,...,NaN,NaN,NaN,NaN,sudhayadav@sansad.nic.in,India,IND,2025,Lok Sabha,bicameral legislature


### Join

In [28]:
no_all = no_parl.merge(no_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
dk_all = dk_parl.merge(dk_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
in_all = in_df.merge(in_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
sg_all = sg_df.merge(sg_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)

bihar_all = bihar_df.merge(bihar_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
tn_all = tn_df.merge(tn_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
up_all = up_df.merge(up_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
hp_all = hp_df.merge(hp_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
del_all = del_df.merge(del_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)

### India is in long. Get it back to MP level

In [29]:
in_grouped = (
    in_all.groupby(['mpsno', 'source_file'], as_index=False)
    .sum(numeric_only=True)
)
in_grouped.head()

,mpsno,source_file,lastloksabha,age,noofterms,numberofsons,numberofdaughters,currentpagenumber,perpagesize,totalelements,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
0,1,ls_14,14,77.0,2.0,2.0,1.0,1,554,554,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,13.0,2.0,0.0
1,2,ls_14,14,53.0,3.0,2.0,0.0,1,554,554,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,13.0,2.0,0.0
2,3,ls_14,14,56.0,2.0,1.0,1.0,1,554,554,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,13.0,2.0,0.0
3,4,ls_15,15,81.0,9.0,1.0,2.0,1,511,511,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,13.0,2.0,0.0
4,5,ls_14,14,74.0,3.0,0.0,2.0,1,554,554,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,13.0,2.0,0.0


In [30]:
in_grouped.shape

(1610, 165)

In [31]:
no_all.describe().round(2)

,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Health insurance information,Passwords,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,...,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0
mean,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
25%,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
50%,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
75%,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
max,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [32]:
dk_all.describe().round(2)

,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Health insurance information,Passwords,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,186.0,186.00,186.0,186.0,186.0,186.00,186.00,186.0,186.0,186.00,...,186.0,186.0,186.0,186.0,186.0,186.0,186.0,186.00,186.0,186.0
mean,2025.0,36.77,0.0,0.0,0.0,3.87,0.97,0.0,0.0,20.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.97,0.0,0.0
std,0.0,6.73,0.0,0.0,0.0,0.71,0.18,0.0,0.0,3.72,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.18,0.0,0.0
min,2025.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
25%,2025.0,38.00,0.0,0.0,0.0,4.00,1.00,0.0,0.0,21.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0
50%,2025.0,38.00,0.0,0.0,0.0,4.00,1.00,0.0,0.0,21.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0
75%,2025.0,38.00,0.0,0.0,0.0,4.00,1.00,0.0,0.0,21.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0
max,2025.0,38.00,0.0,0.0,0.0,4.00,1.00,0.0,0.0,21.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0


In [33]:
in_grouped.groupby(['source_file']).mean()

,mpsno,lastloksabha,age,noofterms,numberofsons,numberofdaughters,currentpagenumber,perpagesize,totalelements,totalpages,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
source_file,,,,,,,,,,,,,,,,,,,,,
ls_13,280.705128,13.000000,76.705128,2.294872,1.467949,1.352564,1.000000,551.000000,551.000000,1.000000,...,0.0,1.000000,1.000000,0.0,0.0,0.0,0.0,13.000000,2.000000,0.0
ls_14,1461.060606,14.636364,77.386364,3.151515,1.590909,1.287879,1.045455,579.181818,579.181818,1.045455,...,0.0,1.045455,1.045455,0.0,0.0,0.0,0.0,13.590909,2.090909,0.0
ls_15,3475.841270,16.666667,74.232804,2.169312,1.391534,1.502646,1.111111,567.777778,567.777778,1.111111,...,0.0,0.952381,0.952381,0.0,0.0,0.0,0.0,12.380952,1.904762,0.0
ls_16,4006.098485,24.787879,99.571970,2.852273,1.837121,1.700758,1.549242,793.212121,793.212121,1.549242,...,0.0,1.549242,1.549242,0.0,0.0,0.0,0.0,20.140152,3.098485,0.0
ls_17,4500.073022,33.103448,113.336714,3.819473,2.206897,1.957404,1.947262,1049.574037,1049.574037,1.947262,...,0.0,1.947262,1.947262,0.0,0.0,0.0,0.0,25.314402,3.894523,0.0
ls_18,5268.476064,32.984043,102.242021,2.555851,1.098404,1.119681,1.832447,996.851064,996.851064,1.832447,...,0.0,1.832447,1.832447,0.0,0.0,0.0,0.0,23.821809,3.664894,0.0


In [34]:
sg_all.describe().round(2)

,leg_start_year,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Health insurance information,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,400.00,400.0,400.00,400.0,400.0,400.0,400.00,400.0,400.0,400.0,...,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.00,400.0,400.0
mean,2011.08,2025.0,6.02,0.0,0.0,0.0,0.22,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.22,0.0,0.0
std,7.35,0.0,11.52,0.0,0.0,0.0,0.41,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.41,0.0,0.0
min,2001.00,2025.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
25%,2006.00,2025.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
50%,2011.00,2025.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
75%,2015.00,2025.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
max,2021.00,2025.0,28.00,0.0,0.0,0.0,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0


In [35]:
bihar_all.describe().round(2)

,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Health insurance information,Passwords,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,...,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0
mean,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
tn_all.describe().round(2)

,photo,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Health insurance information,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,...,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0
mean,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
hp_all.describe().round(2)

,photo,mobile,tele_no_res_no_,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,68.0,6.800000e+01,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,...,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0
mean,0.0,9.416969e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
std,0.0,5.106352e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,7.018000e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
25%,0.0,9.418022e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
50%,0.0,9.418154e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
75%,0.0,9.805725e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
max,0.0,9.882812e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0


In [38]:
up_all.describe().round(2)

,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Health insurance information,Passwords,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,...,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0
mean,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0
25%,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0
50%,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0
75%,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0
max,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0


In [39]:
del_all.describe().round(2)

,ac_no,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Health insurance information,...,Parenting plans,Age groups,Encrypted keys,Races,Spouses names,Audio recordings,Device serial numbers,Salutations,Survey results,Sexual fetishes
count,43.00,43.0,43.00,43.00,43.0,43.0,43.00,43.00,43.0,43.0,...,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.00,43.0,43.0
mean,44.47,2025.0,32.23,0.98,0.0,0.0,2.93,0.98,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.95,0.0,0.0
std,16.26,0.0,5.03,0.15,0.0,0.0,0.46,0.15,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.30,0.0,0.0
min,18.00,2025.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
25%,29.50,2025.0,33.00,1.00,0.0,0.0,3.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00,0.0,0.0
50%,45.00,2025.0,33.00,1.00,0.0,0.0,3.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00,0.0,0.0
75%,58.50,2025.0,33.00,1.00,0.0,0.0,3.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00,0.0,0.0
max,70.00,2025.0,33.00,1.00,0.0,0.0,3.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00,0.0,0.0
